# Job generator for sensitivity tests


In [ ]:
%load_ext dotenv
%dotenv
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from src.config import get_config, get_rng, get_dask_cluster
from typing import List

config = get_config()
cluster, client = get_dask_cluster(config)

In [ ]:
import xarray as xr
import numpy as np

from src.job_generation import (
    Job,
    read_namelist,
    Driver,
    ExperimentCatalog,
    set_radiation_to_dynamic,
    set_initial_soil_conditions_from_precursor,
    set_slurb_deep_soil_temperature_from_dynamic,
    set_spinup_parameters_from_precursor,
    set_surface_pressure_to_dynamic,
    get_nest_mask,
)

In [ ]:
rng = get_rng(config)  # Ensure reproducibility

In [ ]:
jobs: List[Job] = []

## Experiment catalog


In [ ]:
experiments = ExperimentCatalog()
experiments.read_config(Path(config.path.experiments.sensitivity) / "experiments.yml")

## Prepare baseline scenario


In [ ]:
experiments.baseline = Job("slurb_s_base")
experiments.baseline.p3d = read_namelist(
    Path(config.path.experiments.sensitivity) / "base_p3d.yml"
)
experiments.baseline.p3dr = read_namelist(
    Path(config.path.experiments.sensitivity) / "base_p3d.yml"
)
experiments.baseline.p3dr["initialization_parameters"]["initialization_actions"] = (
    "read_restart_data"
)

In [ ]:
experiments.baseline_turbulent_inflow = Path(
    "../../slurb_s_base/INPUT/slurb_s_base_dynamic"
)

In [ ]:
experiments.baseline.register_driver("slurb", Driver())

In [ ]:
experiments.baseline.drivers["slurb"].set_grid("s", vertical=False)
experiments.baseline.drivers["slurb"].set_attrs(
    Path(config.path.experiments.sensitivity) / "global_attributes.yml"
)
experiments.baseline.drivers["slurb"].ds = experiments.baseline.drivers[
    "slurb"
].ds.assign_coords(
    nroof_3d=np.arange(1, 5, dtype=np.int8),
    nroad_3d=np.arange(1, 5, dtype=np.int8),
    nwall_3d=np.arange(1, 5, dtype=np.int8),
    nwindow_3d=np.arange(1, 5, dtype=np.int8),
)

Dynamic driver is used for the soil initial temperature and moisture from the precursor. It is later also used for the inflow boundary condition.


In [ ]:
experiments.baseline.register_driver("dynamic", Driver())
experiments.baseline.drivers["dynamic"].set_grid("s", vertical=False)
experiments.baseline.drivers["dynamic"].set_zsoil()

### Prepare the baseline SLUrb driver

The baseline SLUrb driver is constructed from a YAML config file.


In [ ]:
experiments.baseline.driver_from_config(
    Path(config.path.experiments.sensitivity) / "base_slurb_driver.yml", "slurb"
)

In [ ]:
experiments.baseline.drivers["slurb"]

Cut target urban area from the SLUrb driver. The urban surface covers a 1.5 km x 1.5 km patch in the domain. Covering the whole domain would drastically increase the surface drag compared to the precursor, causing issues with mass balance. A smaller patch also prevents unvanted larger scale feedbacks, keeping the general forcing comparable accross the cases.


In [ ]:
urban_mask = get_nest_mask(
    experiments.baseline.drivers["slurb"].ds["urban_fraction"],
    offset_x=2368.0,
    offset_y=1280.0,
    gridpoints_x=96,
    gridpoints_y=96,
)
for var_name, dataarray in experiments.baseline.drivers["slurb"].ds.data_vars.items():
    if np.issubdtype(dataarray.dtype, np.floating):
        experiments.baseline.drivers["slurb"].ds[var_name] = dataarray.where(
            urban_mask, other=-9999.0
        )
        dataarray.attrs["_FillValue"] = -9999.0
    elif np.issubdtype(dataarray.dtype, np.integer):
        experiments.baseline.drivers["slurb"].ds[var_name] = dataarray.where(
            urban_mask, other=-127
        )
        dataarray.attrs["_FillValue"] = -127
# Urban fraction cannot be fill value
experiments.baseline.drivers["slurb"].ds["urban_fraction"] = (
    experiments.baseline.drivers["slurb"]
    .ds["urban_fraction"]
    .where(experiments.baseline.drivers["slurb"].ds["urban_fraction"] >= 0.0, other=0.0)
)

### Baseline static driver

This is reused from the precursor run, with the exception of setting vegetation type to short grass for the urban area and resetting the roughness back to default.


In [ ]:
experiments.baseline.register_driver("static", Driver())
experiments.baseline.drivers["static"].ds = xr.open_dataset(
    Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "INPUT"
    / "slurb_pre_default_static"
)
experiments.baseline.drivers["static"].set_attrs(
    Path(config.path.experiments.comparison) / "global_attributes.yml"
)
experiments.baseline.drivers["static"].ds["vegetation_type"] = (
    experiments.baseline.drivers["static"]
    .ds["vegetation_type"]
    .where(~urban_mask, other=3)
)
experiments.baseline.drivers["static"].ds = experiments.baseline.drivers[
    "static"
].ds.drop_vars(["vegetation_pars", "nvegetation_pars", "zt"], errors="ignore")
# I don't know why the byte types get casted to into floats when reading the input, but recast them here
int_vars = ("pavement_type", "soil_type", "vegetation_type", "water_type")
experiments.baseline.drivers["static"].ds["pavement_type"][:] = -127
experiments.baseline.drivers["static"].ds["water_type"][:] = -127
for var in int_vars:
    experiments.baseline.drivers["static"].ds[var] = (
        experiments.baseline.drivers["static"].ds[var].astype(np.int8)
    )

### Turbulent inflow


In [ ]:
experiments.baseline.drivers["dynamic"].set_grid("uvws", vertical=True)
experiments.baseline.drivers["dynamic"]

In [ ]:
experiments.baseline.set_turbulent_inflow(
    inflow_source_file=Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "OUTPUT"
    / "slurb_pre_default_yz.001.nc",
    init_source_file=Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "OUTPUT"
    / "slurb_pre_default_3d.001.nc",
    dynamic_source_file=Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "INPUT"
    / "slurb_pre_default_dynamic",
)

In [ ]:
experiments.baseline.drivers["dynamic"]

### Initial soil temperature and moisture

These are averaged from the precursor run after sampled for the corresponding vegetation type used in the main run.


In [ ]:
pre_3d = xr.open_dataset(
    Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "OUTPUT"
    / "slurb_pre_default_3d.001.nc",
    decode_times=False,
)
pre_static = xr.open_dataset(
    Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "INPUT"
    / "slurb_pre_default_static"
)

experiments.baseline.drivers["dynamic"] = set_initial_soil_conditions_from_precursor(
    pre_3d,
    pre_static["vegetation_type"],
    experiments.baseline.drivers["dynamic"],
    experiments.baseline.drivers["static"].ds["vegetation_type"],
)

In [ ]:
# Set corresponding slurb deep soil temp
experiments.baseline.drivers["slurb"] = set_slurb_deep_soil_temperature_from_dynamic(
    experiments.baseline.drivers["slurb"],
    experiments.baseline.drivers["dynamic"],
)

### Spinup mean temperature and amplitude

Quantile difference is used for amplitude for smoothing.


In [ ]:
pre_pr = xr.open_dataset(
    Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "OUTPUT"
    / "slurb_pre_default_pr.001.nc",
    decode_times=False,
)
experiments.baseline.p3d, experiments.baseline.p3dr = (
    set_spinup_parameters_from_precursor(
        pre_pr, experiments.baseline.p3d, experiments.baseline.p3dr
    )
)

### Radiative forcing

This is exactly the same process as for the precursor.


In [ ]:
era5_surf_diurnal = xr.open_dataset(
    config.path.data.raw + "era5/era5_march_2013-2022_surf_diurnal.nc"
)
experiments.baseline.drivers["dynamic"] = set_radiation_to_dynamic(
    era5_surf_diurnal,
    experiments.baseline.drivers["dynamic"],
    time_offset=3 * 3600,
)

### Surface pressure


In [ ]:
experiments.baseline.drivers["dynamic"] = set_surface_pressure_to_dynamic(
    experiments.baseline.drivers["dynamic"], p0=1e5
)

## Create experiments based on experiment configuration


In [ ]:
experiments.baseline.write()

In [ ]:
experiments.generate_experiments()

In [ ]:
for exp in experiments.experiments:
    print(exp)

In [ ]:
# Write job file names for usage in batch run script
with open(Path(config.path.data.jobs) / "sensitivity_test_jobs.txt", "w") as job_file:
    for exp in experiments.experiments:
        job_file.write(exp)
        job_file.write("\n")